In [1]:
# NOTE: ensure pyodbc and ODBC Driver 18 for SQL Server are installed

# Load sample data (dimensions) and save countries as CSV
This notebook inserts small sample data into DimCountry and DimStore, then writes countries to a local CSV file.

In [2]:
# Load configuration from .env file
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
SQL_SERVER_NAME = os.getenv('SQL_SERVER_NAME')
SQL_DATABASE = os.getenv('SQL_DATABASE', 'fabricMirrorDemoDb')
SQL_USER = os.getenv('SQL_USER')
SQL_PASSWORD = os.getenv('SQL_PASSWORD')
LOCAL_OUTPUT_DIR = os.getenv('LOCAL_OUTPUT_DIR', 'data')

# Validate required configuration
if not all([SQL_SERVER_NAME, SQL_USER, SQL_PASSWORD]):
    raise ValueError("Missing required environment variables. Please check your .env file.")
    
# Build full server name
SQL_SERVER = f"{SQL_SERVER_NAME}.database.windows.net"
print(f"Configuration loaded for server: {SQL_SERVER}, database: {SQL_DATABASE}")
print(f"Output directory: {LOCAL_OUTPUT_DIR}")

Configuration loaded for server: sql-mirror-westus3-flopes.database.windows.net, database: fabricMirrorDemoDb
Output directory: c:\VSProjects\Fabric-IPO\data


In [3]:
import os, pyodbc, csv, random, datetime
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)
conn_str = ''.join((
    f"DRIVER={{ODBC Driver 18 for SQL Server}};",
    f"SERVER={SQL_SERVER};",
    f"DATABASE={SQL_DATABASE};",
    f"UID={SQL_USER};",
    f"PWD={SQL_PASSWORD};",
    "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;",
))
conn = pyodbc.connect(conn_str)
cur = conn.cursor()
print('Connected')

Connected


In [4]:
# Upsert helper functions for dimension tables
def upsert_country(code, name, iso3=None):
    cur.execute("SELECT CountryKey FROM dbo.DimCountry WHERE CountryCode = ?", code)
    row = cur.fetchone()
    if row:
        cur.execute("UPDATE dbo.DimCountry SET CountryName = ?, Iso3 = ? WHERE CountryKey = ?", name, iso3, row[0])
    else:
        cur.execute("INSERT INTO dbo.DimCountry (CountryCode, CountryName, Iso3) VALUES (?,?,?)", code, name, iso3)
    conn.commit()

def upsert_product(code, name, category=None, unit_price=None):
    cur.execute("SELECT ProductKey FROM dbo.DimProduct WHERE ProductCode = ?", code)
    row = cur.fetchone()
    if row:
        cur.execute("UPDATE dbo.DimProduct SET ProductName = ?, Category = ?, UnitPrice = ? WHERE ProductKey = ?", name, category, unit_price, row[0])
    else:
        cur.execute("INSERT INTO dbo.DimProduct (ProductCode, ProductName, Category, UnitPrice) VALUES (?,?,?,?)", code, name, category, unit_price)
    conn.commit()

def upsert_store(code, name, country_code, open_date=None):
    cur.execute("SELECT CountryKey FROM dbo.DimCountry WHERE CountryCode = ?", country_code)
    c = cur.fetchone()
    if not c: 
        raise ValueError(f'Country {country_code} not found')
    country_key = c[0]
    cur.execute("SELECT StoreKey FROM dbo.DimStore WHERE StoreCode = ?", code)
    s = cur.fetchone()
    if s:
        cur.execute("UPDATE dbo.DimStore SET StoreName = ?, CountryKey = ?, OpenDate = ? WHERE StoreKey = ?", name, country_key, open_date, s[0])
    else:
        cur.execute("INSERT INTO dbo.DimStore (StoreCode, StoreName, CountryKey, OpenDate) VALUES (?,?,?,?)", code, name, country_key, open_date)
    conn.commit()

In [5]:
# Sample data
countries = [
    ('US', 'United States', 'USA'),
    ('GB', 'United Kingdom', 'GBR'),
    ('FR', 'France', 'FRA'),
    ('DE', 'Germany', 'DEU'),
    ('JP', 'Japan', 'JPN'),
]

products = [
    ('LAPTOP001', 'Business Laptop Pro', 'Electronics', 1299.99),
    ('PHONE001', 'Smartphone X1', 'Electronics', 899.99),
    ('CHAIR001', 'Ergonomic Office Chair', 'Furniture', 349.99),
    ('DESK001', 'Standing Desk Adjustable', 'Furniture', 599.99),
    ('HEADSET001', 'Wireless Headset Premium', 'Electronics', 199.99),
    ('MOUSE001', 'Wireless Mouse Precision', 'Electronics', 79.99),
    ('MONITOR001', '27inch 4K Monitor', 'Electronics', 449.99),
    ('COFFEE001', 'Premium Coffee Beans', 'Food & Beverage', 24.99),
]

stores = [
    ('NY001', 'NYC Downtown', 'US', '2018-01-15'),
    ('LDN01', 'London Central', 'GB', '2019-06-01'),
    ('PAR01', 'Paris Centre', 'FR', '2020-03-20'),
    ('BER01', 'Berlin Mitte', 'DE', '2021-08-05'),
    ('TKY01', 'Tokyo Shibuya', 'JP', '2017-11-10'),
]

In [18]:
# Data loading with execution guard to prevent duplicates
import sys

sys.stdout.flush()

print(f"🚀 Starting data load execution")

# Load countries
print("📍 Loading countries...", end='')
sys.stdout.flush()
for i, c in enumerate(countries):
    upsert_country(*c)
print(f" ✅ {len(countries)} countries loaded")

# Load products  
print("🛍️ Loading products...", end='')
sys.stdout.flush()
for i, p in enumerate(products):
    upsert_product(*p)
print(f" ✅ {len(products)} products loaded")

# Load stores
print("🏪 Loading stores...", end='')
sys.stdout.flush()
for i, s in enumerate(stores):
    code, name, country_code, open_date = s
    upsert_store(code, name, country_code, open_date)
print(f" ✅ {len(stores)} stores loaded")

# Create CSV files
print("📄 Creating CSV exports...")
sys.stdout.flush()

# Countries CSV
csv_path = os.path.join(LOCAL_OUTPUT_DIR, 'countries.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['CountryCode','CountryName','Iso3'])
    for c in countries:
        writer.writerow(c)
print(f"   ✓ Countries: {csv_path}")

# Products CSV
products_csv_path = os.path.join(LOCAL_OUTPUT_DIR, 'products.csv')
with open(products_csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['ProductCode','ProductName','Category','UnitPrice'])
    for p in products:
        writer.writerow(p)
print(f"   ✓ Products: {products_csv_path}")

# Stores CSV
stores_csv_path = os.path.join(LOCAL_OUTPUT_DIR, 'stores.csv')
with open(stores_csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['StoreCode','StoreName','CountryCode','OpenDate'])
    for s in stores:
        writer.writerow(s)
print(f"   ✓ Stores: {stores_csv_path}")

print(f"\n🎉 Data loading execution completed successfully!")
sys.stdout.flush()

🚀 Starting data load execution
📍 Loading countries... ✅ 5 countries loaded
🛍️ Loading products... ✅ 8 products loaded
🏪 Loading stores... ✅ 5 stores loaded
📄 Creating CSV exports...
   ✓ Countries: c:\VSProjects\Fabric-IPO\data\countries.csv
   ✓ Products: c:\VSProjects\Fabric-IPO\data\products.csv
   ✓ Stores: c:\VSProjects\Fabric-IPO\data\stores.csv

🎉 Data loading execution completed successfully!


In [22]:
# Load FactSales with fresh data from the past 24 hours
import sys

print("🧹 Clearing existing FactSales data...")
sys.stdout.flush()

# Delete all existing sales data
cur.execute("DELETE FROM dbo.FactSales")
deleted_count = cur.rowcount
conn.commit()
print(f"   ✓ Deleted {deleted_count} existing sales records")

# Get dimension data for sales generation
print("📊 Loading dimension data...")
sys.stdout.flush()

cur.execute("SELECT StoreKey, CountryKey FROM dbo.DimStore")
store_pairs = cur.fetchall()
cur.execute("SELECT ProductKey, UnitPrice FROM dbo.DimProduct")
product_pairs = cur.fetchall()

if not store_pairs:
    raise RuntimeError('No stores found; ensure dimension tables are loaded.')
if not product_pairs:
    raise RuntimeError('No products found; ensure dimension tables are loaded.')

print(f"   ✓ Found {len(store_pairs)} stores and {len(product_pairs)} products")

def gen_recent_sale():
    """Generate a sale from the past 24 hours"""
    sk, ck = random.choice(store_pairs)
    pk, unit_price = random.choice(product_pairs)
    qty = random.randint(1, 5)
    
    # Convert decimal to float and apply small random variation
    unit_price_float = float(unit_price)
    actual_unit_price = round(unit_price_float * random.uniform(0.9, 1.1), 2)
    amt = round(actual_unit_price * qty, 2)
    
    # Generate sale timestamps from the past 24 hours (1440 minutes)
    minutes_ago = random.randint(0, 1440)  # 0 to 24 hours ago
    seconds_ago = random.randint(0, 59)    # Add random seconds for more precision
    
    sale_time = datetime.datetime.now(datetime.UTC) - datetime.timedelta(minutes=minutes_ago, seconds=seconds_ago)
    
    return sk, ck, pk, sale_time, qty, actual_unit_price, amt

# Generate 10,000 sales records from the past 24 hours
print("🎲 Generating 10,000 sales records from past 24 hours...")
sys.stdout.flush()

batch_size = 1000
total_rows = 10000
inserted_total = 0

# Insert in batches for better performance
for batch_num in range(0, total_rows, batch_size):
    current_batch_size = min(batch_size, total_rows - batch_num)
    rows = [gen_recent_sale() for _ in range(current_batch_size)]
    
    insert_sql = (
        "INSERT INTO dbo.FactSales (StoreKey, CountryKey, ProductKey, SaleTimestamp, Quantity, UnitPrice, Amount) "
        "VALUES (?,?,?,?,?,?,?)"
    )
    
    cur.fast_executemany = True
    cur.executemany(insert_sql, rows)
    conn.commit()
    
    inserted_total += current_batch_size
    print(f"   ✓ Inserted batch {batch_num//batch_size + 1}: {inserted_total:,} / {total_rows:,} records")
    sys.stdout.flush()

# Verify the data
cur.execute("SELECT COUNT(*) FROM dbo.FactSales")
final_count = cur.fetchone()[0]

cur.execute("SELECT MIN(SaleTimestamp), MAX(SaleTimestamp) FROM dbo.FactSales")
time_range = cur.fetchone()
min_time, max_time = time_range

print(f"\n🎉 Sales data loading completed!")
print(f"   📈 Total records in FactSales: {final_count:,}")
print(f"   🕐 Time range: {min_time} to {max_time}")
print(f"   ⏱️  Data spans: {(max_time - min_time).total_seconds() / 3600:.1f} hours")

🧹 Clearing existing FactSales data...
   ✓ Deleted 10000 existing sales records
📊 Loading dimension data...
   ✓ Deleted 10000 existing sales records
📊 Loading dimension data...
   ✓ Found 5 stores and 8 products
🎲 Generating 10,000 sales records from past 24 hours...
   ✓ Found 5 stores and 8 products
🎲 Generating 10,000 sales records from past 24 hours...
   ✓ Inserted batch 1: 1,000 / 10,000 records
   ✓ Inserted batch 1: 1,000 / 10,000 records
   ✓ Inserted batch 2: 2,000 / 10,000 records
   ✓ Inserted batch 2: 2,000 / 10,000 records
   ✓ Inserted batch 3: 3,000 / 10,000 records
   ✓ Inserted batch 3: 3,000 / 10,000 records
   ✓ Inserted batch 4: 4,000 / 10,000 records
   ✓ Inserted batch 4: 4,000 / 10,000 records
   ✓ Inserted batch 5: 5,000 / 10,000 records
   ✓ Inserted batch 5: 5,000 / 10,000 records
   ✓ Inserted batch 6: 6,000 / 10,000 records
   ✓ Inserted batch 6: 6,000 / 10,000 records
   ✓ Inserted batch 7: 7,000 / 10,000 records
   ✓ Inserted batch 7: 7,000 / 10,000 reco